# README
- 페이지 설명 : emotionGenreService 에서 실행하던 코드에 에러가 난 경우 본 페이지에서 수정 후 실전에 적용했습니다.

# 영화 랭킹에서 수집되지 않는 영화들의 코드를 수집하기 (v 해결!)
- 기존 : 네이버 영화 랭킹 사이트에서 순위에 등재되어 있는 영화만 가져왔음
- 변경 : 네이버 검색창에서 직접 request해서 thumbnail tag에 달려있는 영화 코드를 파싱해오기
- 결론 : getNaverMovieInfo 함수를 getAllNaverMovieInfo로 대체

In [7]:
import pandas as pd
cleanedMovie = pd.read_csv('cleaned Movie Title.csv',index_col=0)
reviewYesMovie = pd.read_csv('naverMovieReviewYes.csv',index_col=0)

In [8]:
cleanedMovie.head()

,SRIS_ID,SRIS_NM,WAT_GR_CD,SRIS_NM_CL
0,CS02000001,007 제1탄-살인번호,15,007 살인번호
1,CS02000002,007 제10탄-나를 사랑한 스파이,12,007 나를 사랑한 스파이
2,CS02000003,007 제11탄-문레이커,15,007 문레이커
3,CS02000004,007 제12탄-유어 아이스 온리,15,007 유어 아이스 온리
4,CS02000005,007 제13탄-옥토퍼시,15,007 옥토퍼시


In [10]:
reviewYesMovieList = reviewYesMovie.Title.tolist()
reviewYesMovie.head()

,Score,Title,Code
0,9.59,그린 북,171539
1,9.59,가버나움,174830
2,9.53,베일리 어게인,144906
3,9.49,원더,151196
4,9.49,포드 V 페라리,181710


In [18]:
print('2000개에서 제외된 영화 갯수 : ',len(cleanedMovie['SRIS_NM_CL']))
movieNeedCheck = cleanedMovie[~cleanedMovie['SRIS_NM_CL'].isin(reviewYesMovieList)]
movieNeedCheck.to_csv('movie need to be checked.csv')
movieNeedCheck.head()

2000개에서 제외된 영화 갯수 :  16383


,SRIS_ID,SRIS_NM,WAT_GR_CD,SRIS_NM_CL
0,CS02000001,007 제1탄-살인번호,15,007 살인번호
1,CS02000002,007 제10탄-나를 사랑한 스파이,12,007 나를 사랑한 스파이
2,CS02000003,007 제11탄-문레이커,15,007 문레이커
3,CS02000004,007 제12탄-유어 아이스 온리,15,007 유어 아이스 온리
4,CS02000005,007 제13탄-옥토퍼시,15,007 옥토퍼시


In [20]:
titleList = movieNeedCheck['SRIS_NM_CL'].tolist()

In [62]:
#네이버 검색창에서 리뷰가 정말 없는 영화외에 다 가져올 수 있음
def getAdditionalNaverMovieInfo(titleList : list()) -> dict():
    '''
    input : date, page수
    output : 제목, 코드, 평점
    '''
    from bs4 import BeautifulSoup as bs
    import requests
    
    movieDict = {}
    movieDict['Score'] = []
    movieDict['Title'] = []
    movieDict['Code'] = []
    
    for title in titleList:
        resp= requests.get("https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query="+title)
        soup = bs(resp.text, "html.parser")
        
        #영화 평점, 없으면 0으로 인식
        try:
            movieDict['Score'].append(soup.select('dl.r_grade > dd > em')[0].text)
        except:
            movieDict['Score'].append(0)
        
        #영화 제목 
        movieDict['Title'].append(title)
        
        #영화 코드
        try:
            movieDict['Code'].append(soup.select('div.thumb > a')[0].attrs['href'].split('code=')[1])
        except:
            movieDict['Code'].append('nan')
            
        #print(movieDict)

    return movieDict

In [61]:
movieCode = getAllNaverMovieInfo(titleList)

KeyboardInterrupt: 

해결!

# 네이버 영화 리뷰 수집 (v 해결!)
- 기존 : iframe 부분에 있는 데이터를 수집할 수 없는 코드
- 변경 : selenium을 이용해서 iframe으로 접속해서 크롤링, 한줄평과 element 까지 가져오기
- 결론 : getNaverMovieCommentText 함수를 getNaverMovieCommentTextIframe 함수로 대체

In [178]:
def getNaverMovieCommentElementTextIframe(codeList : list())-> pd.DataFrame():
    '''
    input : 영화 코드 리스트
    output : 코드, 한줄평 
    '''
    from bs4 import BeautifulSoup as bs
    import time 
    from selenium import webdriver
    import requests

    #load driver
    driver = webdriver.Chrome(r'C:\Users\owner\hyesung\2020상반기 SK브로드밴드 인턴\CRM기획팀\chromedriver_win32\chromedriver')
    
    #데이터 담을 그릇
    dataDict = dict()
    dataDict['Code'] = []
    dataDict['Element'] = []
    dataDict['Comment'] = []
    
    for code in codeList:
        dataDict['Code'].append(code) #code 
    
        url = 'https://movie.naver.com/movie/bi/mi/point.nhn?code='+str(code)
        driver.get(url)   # 대상 사이트 진입 
        
        ##감상 요소 크롤링
        soupOrigin = bs(driver.page_source)
        elementDict = dict()
        for li in soupOrigin.select('div.graph_box > div.grp_sty4 > ul.grp_point > li'):
            #print(li.select('span.grp_score')[0].text.split('%')[0])
            element, ratio = li.select('strong')[0].text, int(li.select('span.grp_score')[0].text.split('%')[0])
            elementDict[element] = ratio
            
        elementList = sorted(elementDict.items(), key = lambda x: x[1], reverse= True) # 네티즌 투표수(투표 비중) 기준으로 정렬
        
        elementSelected = str() 
        for e in elementList[:2]: #top2 요소만 선별
            elementSelected += e[0]+' ' #요소 + 요소 문자형으로 저장
            
        dataDict['Element'].append(elementSelected) #선별된 element    
        
        ##iframe 부분 크롤링 : 한줄평
        driver.switch_to_frame("pointAfterListIframe")    # pointAfterListIframe으로 스위칭 
        soupIframe = bs(driver.page_source)

        comments = str()
        for i, li in enumerate(soupIframe.select('div.score_result > ul >li')):
            
            #print(li.select_one('div.score_reple > p > span#_filtered_ment_'+str(i)))
            comments += li.select_one('div.score_reple > p > span#_filtered_ment_'+str(i)).text.strip() + '\n'  #\n을 구분자로 모든 한줄평을 이어 붙임

        dataDict['Comment'].append(comments)
        
    driver.quit()
    return dataDict

In [176]:
tmp = getNaverMovieCommentElementTextIframe([36734, 185917])

c:\users\owner\anaconda3\envs\dash_board\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: use driver.switch_to.frame instead


In [177]:
pd.DataFrame(tmp)

,Code,Element,Comment
0,36734,연기 스토리,케이트 허드슨 진상연기 완전 웃김 ㅋㅋㅋㅋㅋㅋ 재밌게봤어요ㅋㅋㅋ\n케이트허드슨 존예...
1,185917,연출 연기,재미있게 잘 봤습니다. 다만 조금 질질 끄는 게 아쉬웠어요. 전형적인 약간의 억지 ...


결론 : 해결!

# code가 수집되지 않은 영화 (v 해결)
- 원인 : html tag 구조가 다름
- 결론: getErrorNaverMovieInfo 함수 사용해서 해결

In [189]:
codeNoneList = pd.read_csv('code none movie.csv',index_col= 0)['Title'].tolist()

In [190]:
codeNoneList

['퍼햅스 러브',
 '선물',
 '동화',
 '더 파티',
 '드리프트',
 '앨리스 죽이기',
 '아르마딜로',
 '메기의 추억',
 '마이 플레이스',
 '타샤 튜더',
 '냉혈인',
 '편지',
 '바우하우스',
 '너와 나의 21세기',
 '외로운 산장에서',
 '절대쌍교',
 '스퀴즈',
 '소중한사람',
 '블루',
 '한나 아렌트',
 '클라라',
 '타임라인',
 '미스터 홈즈',
 '지금 이 순간',
 '슈퍼소닉',
 '스페셜 솔져',
 '5쿼터',
 '비스트',
 '롤러코스터',
 '설',
 '밀양',
 '오월애',
 '피해자',
 '매니페스토',
 '바라나시',
 '삼례',
 '빅 화이트',
 '노스페이스',
 '해운대',
 '런던 타운',
 '나를 찾아서',
 '파이판',
 '소중한 사람',
 '이조 상노비사',
 '나비잠',
 '버터 러버',
 '테스',
 '디터 람스',
 '나는 나비',
 '어쌔신 크리드',
 '라 파미에',
 '다이애나',
 '마테호른',
 '비 오는 날 수채화',
 '코알라',
 '다잉',
 '주리',
 '나는 고양이로소이다',
 '격전',
 '헷지',
 '모정에 우는 두 아들',
 '룸바',
 '메이트',
 '블러바드',
 '명동에서 첫사랑을',
 '세계일주',
 'Skyscraper',
 '눈물',
 '박서방',
 'The Little Prince',
 '10대의 반항',
 '버킷 리스트',
 '셧업',
 '개와 늑대 사이의 시간',
 '부활',
 '난중일기',
 '노라노',
 '릴라 릴라',
 '미친거아니야',
 '태극권',
 '무단침입',
 '몽골',
 '나자리노',
 '제니스 웨딩',
 '휴가',
 '나의 라임 오렌지 나무',
 '해저 2만리',
 '어느 여대생의 고백',
 '박대박',
 '김광석',
 '검은 여름',
 '기생충 : 흑백판',
 '경복',
 '미스터 쉐프',
 '촌철살인',
 '몽중인',
 '사랑이 이긴다',
 '현기증',
 '하나식당',
 '레오나르도 다 빈치',
 '샘터분식',
 '물

In [258]:
#네이버 검색창에서 리뷰가 정말 없는 영화외에 다 가져올 수 있음
def getErrorNaverMovieInfo(titleList : list()) -> dict():
    '''
    input : date, page수
    output : 제목, 코드, 평점
    '''
    from bs4 import BeautifulSoup as bs
    import requests
    
    movieDict = {}
    movieDict['Score'] = []
    movieDict['Title'] = []
    movieDict['Code'] = []
    
    for title in titleList:
        resp= requests.get("https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query="+title)
        soup = bs(resp.text, "html.parser")
        
        #영화 평점, 없으면 0으로 인식
        try:
            movieDict['Score'].append(soup.select('dl.r_grade > dd > em')[0].text)
        except:
            movieDict['Score'].append(0)
        
        #영화 제목 
        movieDict['Title'].append(title)
        
        #영화 코드
        try:
            movieDict['Code'].append(soup.select_one('div.info_main').select_one('a.sp_thmb').attrs['href'].split('code=')[1])
        except:
            #print(soup.select_one('div.info_main').select_one('h3').select_one('a.sh_movie_link').attrs['href'].split('code=')[1])
            try:
                movieDict['Code'].append(soup.select_one('div.info_main').select_one('h3').select_one('a.sh_movie_link').attrs['href'].split('code=')[1])
            except:
                movieDict['Code'].append('nan')
                print(title, '넌 대체 뭐하는 놈이냐')
        #print(movieDict)

    return movieDict

In [259]:
codeErrorMovie = getErrorNaverMovieInfo(codeNoneList)

앨리스 죽이기 넌 대체 뭐하는 놈이냐
간디 넌 대체 뭐하는 놈이냐
반달곰 넌 대체 뭐하는 놈이냐


In [260]:
tmp = pd.DataFrame(codeErrorMovie)
tmp.to_csv('code error movie complete.csv')
tmp.head()

,Score,Title,Code
0,6.51,퍼햅스 러브,43254
1,8.96,선물,31118
2,9.00,동화,170311
3,8.29,더 파티,159110
4,8.15,드리프트,96044


In [ ]:
43254

결론 :해결!

# 한줄평이 있는데도 수집되지 않는 영화 (v 해결)
- 원인 : 잘못된 code가 수집, selenium 자체 에러, 네트워크 불안정성
- 결론 : 예외가 발생한 영화를 뽑아서 추가적으로 진행

# 명작인데 누락된 영화들 원인 파악 (v 해결)
- 원인 : 19금이 아닌데 19금으로 되어있는 데이터 문제
- 해결 : 19금으로 분류된 영화중에서 눈으로 훑어보고 아니다 싶은 것은 확인 후 추가하는 방안

In [5]:
import pandas as pd
goodMovie = pd.read_csv('cleaned Movie Title.csv',index_col=0)
goodMovie

,SRIS_ID,SRIS_NM,WAT_GR_CD,SRIS_NM_CL
0,CS02000001,007 제1탄-살인번호,15,007 살인번호
1,CS02000002,007 제10탄-나를 사랑한 스파이,12,007 나를 사랑한 스파이
2,CS02000003,007 제11탄-문레이커,15,007 문레이커
3,CS02000004,007 제12탄-유어 아이스 온리,15,007 유어 아이스 온리
4,CS02000005,007 제13탄-옥토퍼시,15,007 옥토퍼시
...,...,...,...,...
22008,CS11032315,진정령 생혼,15,진정령 생혼
22009,CS11032318,알.이.엠[캐치온],15,알.이.엠
22010,CS11032324,무협: 더 코드,15,무협: 더 코드
22011,CS11032383,극장판 반지의 비밀일기[별도편성][예고편],0,극장판 반지의 비밀일기


In [9]:
goodMovie = goodMovie.dropna()
goodMovie[goodMovie['SRIS_NM_CL'].str.contains('부다')]

,SRIS_ID,SRIS_NM,WAT_GR_CD,SRIS_NM_CL
12946,CS02017552,이동진의 라이브톡 : 그랜드 부다페스트 호텔,12,이동진의 라이브톡 : 그랜드 부다페스트 호텔
13143,CS02018431,부다페스트 느와르,15,부다페스트 느와르
17076,CS11006007,CGV아트하우스 이동진의 라이브톡 : 그랜드 부다페스트 호텔,12,CGV아트하우스 이동진의 라이브톡 : 그랜드 부다페스트 호텔
18140,CS11012104,부다페스트 로큰롤,15,부다페스트 로큰롤
21017,CS11027494,갈까부다,12,갈까부다
